In [1]:
###### modules pour le chargement des données depuis le XML ######
import glob
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

###### modules pour la classification ######

# modèles
from sklearn.svm import LinearSVC, SVC

# vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

# création de nos transformers
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer # créer nos propres transformer

# recherche des meilleurs hyperparamètres
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# résultats
from sklearn.metrics import classification_report

# sauvegarde des modèles
from joblib import dump, load

###### modules pour la visualisation ######
import matplotlib.pyplot as plt
import pandas as pd

###### miscellaneous ######
from typing import List # typage des fonctions
import numpy as np
import re
from collections import namedtuple

##### multi class #####
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

# Glove vector 

In [ ]:
import glob
import os
#root folder
root_folder='.'
glove_filename='vectors.txt'

# Variable for data directory
glove_path = os.path.abspath(glove_filename)


from gensim.models import KeyedVectors
word2vec_output_file = glove_filename+'.word2vec'
glove = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

#remplace un CountVectorizer()
def GloveVectorizer(X_train):
    n=0
    m = glove.get_vector('président')
    docs = np.zeros((len(X_train),m.shape[0]))
    for document in X_train:
        word_vectors=[]
        for token in document.split():
            if token in glove:
                word_vectors.append(glove[token])
        if len(word_vectors)>0:
            word_vectors = np.array(word_vectors)
            docs[n]=word_vectors.mean(axis=0)
        else:
            docs[n]=np.zeros(m.shape[0])              
        n=n+1
    return docs

# w2vec vector

In [ ]:
w2v_filename='text8-vector.bin'

# Variable for data directory
w2v_path = os.path.abspath(w2v_filename)

wv_from_bin = KeyedVectors.load_word2vec_format(w2v_path, binary=True)
def w2vVectorizer(X_train):
    n=0
    m = wv_from_bin.get_vector('président')
    docs = np.zeros((len(X_train),m.shape[0]))
    for document in X_train:
        word_vectors=[]
        for token in document.split():
            if token in wv_from_bin:
                word_vectors.append(wv_from_bin[token])
        if len(word_vectors)>0:
            word_vectors = np.array(word_vectors)
            docs[n]=word_vectors.mean(axis=0)
        else:
            docs[n]=np.zeros(m.shape[0])          
        n=n+1
    return docs

# FastText vector

Les résultats sont pas meilleurs que pour w2v, je vous mets quand même ce que j'ai fait. J'ai pas pu importer les vecteurs parce que trop lourd.

```python
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_vectors
fde = 'cc.fr.300.bin'
ft_path = os.path.abspath(fde)
print(ft_path)
ft = load_facebook_vectors(fde)
```

```python
def FTVectorizer(X_train):
    n=0
    m = ft.get_vector('président')
    docs = np.zeros((len(X_train),m.shape[0]))
    for document in X_train:
        word_vectors=[]
        for token in document.split():
            if token in ft:
                word_vectors.append(ft[token])
        if len(word_vectors)>0:
            word_vectors = np.array(word_vectors)
            docs[n]=word_vectors.mean(axis=0)
        else:
            docs[n]=np.zeros(m.shape[0])              
        n=n+1
    return docs
```

# Les meilleures pipelines 

## Random Forest Classifier

In [ ]:
pipeline_r = Pipeline([
        ('ngram_tf_idf', Pipeline([
          ('counts', FunctionTransformer(w2vVectorizer)),
          ('tf_idf', TfidfTransformer(use_idf=False))
        ])),
  ('standard', StandardScaler(with_mean=False)),
  ('r',RandomForestClassifier(criterion='gini',max_features='sqrt'))])


liste_rfc.append(pipeline_r)

# SGD 

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(CountVectorizer(),TfidfTransformer(),StandardScaler(with_mean=False),
                    OneVsOneClassifier(SGDClassifier(max_iter=1000, tol=1e-3)))

